In [1]:
import requests

def download_html(url, output_file):
    """
    Download HTML content of a given URL and save it to a file.
    """
    try:
        print(f"Fetching: {url}")
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise an exception for HTTP errors
        html_content = response.text
        
        # Save HTML to file
        with open(output_file, "w", encoding="utf-8") as file:
            file.write(html_content)
        
        print(f"HTML saved to {output_file}")
    except requests.RequestException as e:
        print(f"Failed to fetch {url}: {e}")


# Replace with the URL you want to test
test_url = "https://www.yourlifechiro.com/articles/general/page/1"
output_filename = "test_page_1.html"

download_html(test_url, output_filename)


Fetching: https://www.yourlifechiro.com/articles/general/page/1
HTML saved to test_page_1.html


In [2]:
import sqlite3
import requests
from time import sleep
from tqdm import tqdm

# Database setup
DB_NAME = 'webscraping.db'
TABLE_NAME = 'yellow_pages'

def initialize_database():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            link TEXT UNIQUE,
            html TEXT,
            state INTEGER DEFAULT 0
        )
    """)
    conn.commit()
    conn.close()

def populate_jobs():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    base_url = "https://www.yourlifechiro.com/articles/general/page/"
    for i in range(1, 37):  # Page 1 to 36
        link = f"{base_url}{i}"
        cursor.execute(f"INSERT OR IGNORE INTO {TABLE_NAME} (link) VALUES (?)", (link,))
    
    conn.commit()
    conn.close()

def fetch_and_update_html():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Fetch jobs where state = 0
    cursor.execute(f"SELECT id, link FROM {TABLE_NAME} WHERE state = 0")
    jobs = cursor.fetchall()

    for job in tqdm(jobs, desc="Scraping Pages"):
        job_id, link = job
        try:
            response = requests.get(link, timeout=10)
            response.raise_for_status()  # Raise an error for HTTP issues
            html_content = response.text
            
            # Update the database with the HTML and change state to 1
            cursor.execute(f"""
                UPDATE {TABLE_NAME}
                SET html = ?, state = 1
                WHERE id = ?
            """, (html_content, job_id))
            conn.commit()
        except requests.RequestException as e:
            print(f"Failed to fetch {link}: {e}")
        
        # Add a small delay to avoid hammering the server
        sleep(1)

    conn.close()

initialize_database()
populate_jobs()
fetch_and_update_html()


Scraping Pages: 100%|██████████| 36/36 [01:03<00:00,  1.77s/it]


In [4]:
import sqlite3
from bs4 import BeautifulSoup

# Database setup
DB_NAME = 'webscraping.db'
TABLE_YELLOW_PAGES = 'yellow_pages'
TABLE_ARTICLES = 'articles'

def initialize_database():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Add a new column to yellow_pages for the count of articles
    cursor.execute(f"ALTER TABLE {TABLE_YELLOW_PAGES} ADD COLUMN number_of_articles INTEGER DEFAULT 0")

    # Create articles table
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {TABLE_ARTICLES} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            parent_link TEXT,
            article_html TEXT,
            FOREIGN KEY (parent_link) REFERENCES {TABLE_YELLOW_PAGES} (link)
        )
    """)
    
    conn.commit()
    conn.close()

def parse_and_store_articles():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Fetch all HTML with state 1
    cursor.execute(f"SELECT id, link, html FROM {TABLE_YELLOW_PAGES} WHERE state = 1")
    rows = cursor.fetchall()

    for row in rows:
        page_id, link, html = row

        # Parse HTML using BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')
        articles = soup.select('li.articles__item')
        
        # Insert articles into the articles table
        for article in articles:
            cursor.execute(f"""
                INSERT INTO {TABLE_ARTICLES} (parent_link, article_html)
                VALUES (?, ?)
            """, (link, article.encode(formatter=None)))

        # Update yellow_pages with number_of_articles and change state to 2
        cursor.execute(f"""
            UPDATE {TABLE_YELLOW_PAGES}
            SET number_of_articles = ?, state = 2
            WHERE id = ?
        """, (len(articles), page_id))

        conn.commit()
    
    conn.close()

# Ensure the database is updated with the necessary tables and columns
initialize_database()

# Parse HTML and store articles
parse_and_store_articles()

print("Processing complete.")


Processing complete.


In [ ]:
import sqlite3
import pandas as pd

# Database file name
DB_NAME = 'webscraping.db'
TABLE_NAME = 'articles'

def load_yellow_pages_as_dataframe():
    # Connect to SQLite database
    conn = sqlite3.connect(DB_NAME)
    
    # Load the yellow_pages table into a DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {TABLE_NAME}", conn)
    
    # Close the database connection
    conn.close()
    
    return df

# Load the table as a DataFrame
df = load_yellow_pages_as_dataframe()

# Display the DataFrame
df.to_excel('output.xlsx', index=False)

In [ ]:
import sqlite3
from bs4 import BeautifulSoup

# Database setup
DB_NAME = 'webscraping.db'
TABLE_ARTICLES = 'articles'

def add_columns_to_articles():
    """
    Add new columns to the articles table for storing parsed data.
    """
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Add new columns for title, URL, category, and content if they don't already exist
    cursor.execute("ALTER TABLE articles ADD COLUMN title TEXT")
    cursor.execute("ALTER TABLE articles ADD COLUMN url TEXT")
    cursor.execute("ALTER TABLE articles ADD COLUMN category TEXT")
    cursor.execute("ALTER TABLE articles ADD COLUMN content TEXT")

    conn.commit()
    conn.close()

def parse_article_html():
    """
    Parse the HTML of each article and extract title, URL, category, and content.
    """
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Fetch all articles that haven't been parsed (i.e., where title is NULL)
    cursor.execute(f"SELECT id, article_html FROM {TABLE_ARTICLES} WHERE title IS NULL")
    rows = cursor.fetchall()

    for row in rows:
        article_id, article_html = row
        try:
            # Parse the HTML using BeautifulSoup
            soup = BeautifulSoup(article_html, 'html.parser')

            # Extract the title
            title_tag = soup.select_one('a.article__post-title')
            title = title_tag.text.strip() if title_tag else None

            # Extract the URL
            url = title_tag['href'] if title_tag and 'href' in title_tag.attrs else None

            # Extract the category
            category_tag = soup.select_one('span.article__post-category')
            category = category_tag.text.replace('Category:', '').strip() if category_tag else None

            # Extract the content
            content_tag = soup.select_one('div.article__post-content p')
            content = content_tag.text.strip() if content_tag else None

            # Update the database with the parsed data
            cursor.execute(f"""
                UPDATE {TABLE_ARTICLES}
                SET title = ?, url = ?, category = ?, content = ?
                WHERE id = ?
            """, (title, url, category, content, article_id))
            
            conn.commit()
        except Exception as e:
            print(f"Failed to parse article ID {article_id}: {e}")

    conn.close()

# Add necessary columns to the articles table
add_columns_to_articles()

# Parse and extract details for each article
parse_article_html()

print("Parsing complete.")


Parsing complete.


In [1]:
import sqlite3
import requests
from time import sleep
from tqdm import tqdm

# Database setup
DB_NAME = 'webscraping.db'
TABLE_ARTICLES = 'articles'

def add_columns_to_articles_for_request():
    """
    Add new columns to the articles table for storing state and HTML content.
    """
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Add new columns for state and content_html if they don't already exist
    cursor.execute("ALTER TABLE articles ADD COLUMN state INTEGER DEFAULT 0")
    cursor.execute("ALTER TABLE articles ADD COLUMN content_html TEXT")

    conn.commit()
    conn.close()

def fetch_article_html():
    """
    Fetch HTML content for each article's URL and update the database.
    """
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Fetch all articles with state = 0 (pending)
    cursor.execute(f"SELECT id, url FROM {TABLE_ARTICLES} WHERE state = 0")
    articles = cursor.fetchall()

    for article in tqdm(articles, desc="Fetching Articles"):
        article_id, url = article
        try:
            # Request the URL
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Raise an exception for HTTP errors
            content_html = response.text

            # Update the database with the fetched HTML and set state to 1 (success)
            cursor.execute(f"""
                UPDATE {TABLE_ARTICLES}
                SET content_html = ?, state = 1
                WHERE id = ?
            """, (content_html, article_id))

        except requests.RequestException as e:
            # Update the state to 2 (error) in case of failure
            print(f"Failed to fetch article ID {article_id} ({url}): {e}")
            cursor.execute(f"""
                UPDATE {TABLE_ARTICLES}
                SET state = 2
                WHERE id = ?
            """, (article_id,))

        # Commit changes after each article to ensure persistence
        conn.commit()

        # Add a delay to prevent hammering the server
        sleep(1)

    conn.close()

if __name__ == "__main__":
    # Add necessary columns to the articles table
    add_columns_to_articles_for_request()

    # Fetch HTML content for each article
    fetch_article_html()

    print("Article fetching complete.")


Fetching Articles: 100%|██████████| 570/570 [18:01<00:00,  1.90s/it]

Article fetching complete.


In [3]:
import sqlite3
import os
import uuid

# Database setup
DB_NAME = 'webscraping.db'
TABLE_ARTICLES = 'articles'
OUTPUT_FOLDER = 'html_exports'

def save_html_files():
    """
    Save all content_html entries from the articles table into .html files.
    """
    # Ensure the output folder exists
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    # Connect to the database
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Fetch all rows where content_html is not NULL
    cursor.execute(f"SELECT content_html FROM {TABLE_ARTICLES} WHERE content_html IS NOT NULL")
    rows = cursor.fetchall()

    for row in rows:
        content_html = row[0]
        try:
            # Generate a unique filename using UUID
            file_name = f"{uuid.uuid4()}.html"
            file_path = os.path.join(OUTPUT_FOLDER, file_name)

            # Save the HTML content to the file
            with open(file_path, "w", encoding="utf-8") as file:
                file.write(content_html)

            print(f"Saved: {file_path}")
        except Exception as e:
            print(f"Failed to save HTML: {e}")

    # Close the database connection
    conn.close()

if __name__ == "__main__":
    save_html_files()
    print(f"All HTML files have been saved to the '{OUTPUT_FOLDER}' folder.")


Saved: html_exports\f477d47f-bbfb-4629-b450-fd7f99f5b2ff.html
Saved: html_exports\8a8ef95f-b59a-4a2d-863b-7f9f69e583b6.html
Saved: html_exports\a5196b61-a15f-47db-8f85-f05a71cf6c31.html
Saved: html_exports\e36daa4f-0265-4fd6-9bce-6143121a044f.html
Saved: html_exports\328fde2d-f6fd-4782-9c9f-cda98090b58a.html
Saved: html_exports\875c7dc7-03cd-4f50-89e6-747902657440.html
Saved: html_exports\3a3ab23c-9e94-406c-8854-b21999a2cc89.html
Saved: html_exports\af1b7803-fcdb-4b2f-a060-92c5e24653c4.html
Saved: html_exports\29370018-bed9-46ce-be68-c6e4cefd53f2.html
Saved: html_exports\d3b74ce2-842d-43d4-af4a-0a33e8d35993.html
Saved: html_exports\ab133ee1-b90d-4dd8-b127-3e385a22695d.html
Saved: html_exports\28e2adfc-94ba-47d5-83ac-231b08dae139.html
Saved: html_exports\32f3237a-cded-44ee-82cd-af5fde6ae2d7.html
Saved: html_exports\63d273ce-9038-4fcd-b1c7-c74264e08b73.html
Saved: html_exports\281fc8f2-f544-4d26-9259-6b20bdb7746b.html
Saved: html_exports\e40b89c3-39a5-46ee-b2c0-fd04507d3348.html
Saved: h

In [4]:
import sqlite3
from bs4 import BeautifulSoup


def add_columns_to_articles_for_parsed_data():
    """
    Add new columns to store blog_title, created_in, and content in the articles table.
    """
    conn = sqlite3.connect('webscraping.db')
    cursor = conn.cursor()

    # Add new columns for parsed data if they don't already exist
    cursor.execute("ALTER TABLE articles ADD COLUMN blog_title TEXT")
    cursor.execute("ALTER TABLE articles ADD COLUMN created_in TEXT")
    cursor.execute("ALTER TABLE articles ADD COLUMN content TEXT")

    conn.commit()
    conn.close()


def parse_and_update_articles():
    """
    Parse the content_html of each article to extract blog_title, created_in, and content (with images).
    Update the articles table with the parsed data.
    """
    conn = sqlite3.connect('webscraping.db')
    cursor = conn.cursor()

    # Fetch all articles with non-null content_html that haven't been parsed (blog_title is NULL)
    cursor.execute("SELECT id, content_html FROM articles WHERE blog_title IS NULL")
    articles = cursor.fetchall()

    for article in articles:
        article_id, content_html = article

        try:
            # Parse the HTML using BeautifulSoup
            soup = BeautifulSoup(content_html, 'html.parser')

            # Extract the blog title
            blog_title_tag = soup.select_one('div.article h2')
            blog_title = blog_title_tag.text.strip() if blog_title_tag else None

            # Extract the created in categories
            created_in_tag = soup.select_one('div.article__info-list li.article__info-item')
            created_in = created_in_tag.text.replace("Created in", "").strip() if created_in_tag else None

            # Extract the content (including images as HTML)
            content_tag = soup.select_one('div.article__content')
            content = str(content_tag) if content_tag else None  # Keep the entire content HTML, including images

            # Update the articles table
            cursor.execute("""
                UPDATE articles
                SET blog_title = ?, created_in = ?, content = ?
                WHERE id = ?
            """, (blog_title, created_in, content, article_id))

            conn.commit()
        except Exception as e:
            print(f"Failed to parse article ID {article_id}: {e}")

    conn.close()


if __name__ == "__main__":
    # Add necessary columns to the articles table
    add_columns_to_articles_for_parsed_data()

    # Parse and update articles with extracted data
    parse_and_update_articles()

    print("Parsing and updating articles complete.")


OperationalError: duplicate column name: content

In [ ]:
import sqlite3
from bs4 import BeautifulSoup

# Database setup
DB_NAME = 'webscraping.db'
TABLE_ARTICLES = 'articles'

def add_columns_for_blog_details():
    """
    Add new columns to the articles table for storing parsed blog details.
    Handles cases where the columns already exist.
    """
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # List of new columns to add
    new_columns = {
        "blog_title": "TEXT",
        "created_in": "TEXT",
        "blog_content_html": "TEXT",
        "breadcrumb": "TEXT"
    }

    for column, column_type in new_columns.items():
        try:
            cursor.execute(f"ALTER TABLE {TABLE_ARTICLES} ADD COLUMN {column} {column_type}")
        except sqlite3.OperationalError as e:
            if "duplicate column name" in str(e).lower():
                print(f"Column {column} already exists, skipping.")
            else:
                raise

    conn.commit()
    conn.close()

def parse_blog_details():
    """
    Parse HTML content from the articles table and extract blog details.
    """
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Fetch all articles where blog_title is NULL (indicating unprocessed rows)
    cursor.execute(f"SELECT id, content_html FROM {TABLE_ARTICLES}")
    articles = cursor.fetchall()

    for article in articles:
        article_id, content_html = article

        try:

            soup = BeautifulSoup(content_html, 'html.parser')

            # Extract blog title
            title_tag = soup.select_one('h2')
            blog_title = title_tag.text.strip() if title_tag else None

            # Extract created_in
            created_in_tag = soup.select_one('.article__info .article__info-list .article__info-item')
            created_in = created_in_tag.text.strip() if created_in_tag else None

            # Extract blog_content_html
            content_tag = soup.select_one('.article__content')
            blog_content_html = str(content_tag) if content_tag else None

            # Extract breadcrumb
            breadcrumb_tag = soup.select_one('.articles__breadcrumbs')
            breadcrumb = str(breadcrumb_tag) if breadcrumb_tag else None

            # Update the database with parsed data
            cursor.execute(f"""
                UPDATE {TABLE_ARTICLES}
                SET blog_title = ?, created_in = ?, blog_content_html = ?, breadcrumb = ?
                WHERE id = ?
            """, (blog_title, created_in, blog_content_html, breadcrumb, article_id))

            conn.commit()

        except Exception as e:
            print(f"Failed to parse article ID {article_id}: {e}")

    conn.close()

if __name__ == "__main__":
    # Add necessary columns to the articles table
    add_columns_for_blog_details()

    # Parse and extract blog details
    parse_blog_details()

    print("Blog parsing complete.")


Column blog_title already exists, skipping.
Column created_in already exists, skipping.
Column blog_content_html already exists, skipping.
Column breadcrumb already exists, skipping.
<ul class="articles__breadcrumbs" style="overflow-x:hidden; ">
<li style="display:inline">
<a href="https://www.yourlifechiro.com">Home</a> <span class="breadcrumbs-separator">&gt;</span>
</li>
<li style="display:inline">
<a href="https://www.yourlifechiro.com/articles/general">Articles</a> <span class="breadcrumbs-separator">&gt;</span>
</li>
<li style="display:inline">
<a href="https://www.yourlifechiro.com/articles/general/category/39807">Newsletter Library</a> <span class="breadcrumbs-separator">&gt;</span>
</li>
<li style="display:inline">
            	November Newsletter: The Benefits of Spinal Decompression Therapy
            </li>
</ul>
<ul class="articles__breadcrumbs" style="overflow-x:hidden; ">
<li style="display:inline">
<a href="https://www.yourlifechiro.com">Home</a> <span class="breadcrumb

In [25]:
import sqlite3
import pandas as pd

# Database file name
DB_NAME = 'webscraping.db'
TABLE_NAME = 'articles'

def load_yellow_pages_as_dataframe():
    # Connect to SQLite database
    conn = sqlite3.connect(DB_NAME)
    
    # Load the yellow_pages table into a DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {TABLE_NAME}", conn)
    
    # Close the database connection
    conn.close()
    
    return df

# Load the table as a DataFrame
df = load_yellow_pages_as_dataframe()

# Display the DataFrame
df.to_excel('output.xlsx', index=False)